In [ ]:
execfile('d:/measuring/analysis/scripts/setup_analysis.py')
%matplotlib inline
import matplotlib
import time

# analyse data tracked during step & stabilize measurement

In [ ]:
folder= tb.latest_data('step_and_stable',older_than=None)#,older_than='20171228155500')#r'D:\measuring\data\20171207\135353_pidcavl'
#r'D:\measuring\data\20171207\133039_pidcavl'#r'D:\measuring\data\20171207\130217_pidcavl'#r'D:\measuring\data\20171206\171035_pidcavl'
d = np.loadtxt(tb.get_msmt_fp(folder, ext='dat'), skiprows=14)

modulation_freq = 200 #Hz
modulation_bins = 200

time, jpe, error, avg_readout = d[:,0],d[:,1],d[:,2],d[:,3]


# time = time + 17*3600+51*60+8

In [ ]:
fig,ax=plt.subplots(1, figsize=(16,5))
ax.set_title(folder)
ax.plot(time,jpe,'-',color='red')
ax.set_xlabel("time (seconds)", fontsize = 14)
ax.set_ylabel("jpe voltage (V)", fontsize = 14, color ='r')
ax2 =ax.twinx()
# ax.set_ylim(0,300000)
# ax.set_xlim(200,400)
# ax2.set_xlim(6,11)
ax2.plot(time,avg_readout*modulation_freq)
ax2.set_ylabel('avg_readout (Hz)', color = 'b',fontsize=14)
# ax.set_xlim(150,200)
plt.savefig(os.path.join(folder,'overlap_wm_avgcts_plot.png'))

In [ ]:
fig,ax=plt.subplots(1, figsize=(16,5))
ax.set_title(folder)
ax.plot(time,jpe,'-',color='red')
ax.set_xlabel("time (seconds)", fontsize = 14)
ax.set_ylabel("jpe voltage (V)", fontsize = 14, color ='r')
ax2 =ax.twinx()
# ax.set_ylim(0,300000)
# ax.set_xlim(200,400)
# ax2.set_xlim(6,11)
ax2.plot(time,error)
ax2.set_ylabel('error signal', color = 'b',fontsize=14)
# ax2.set_ylim(-2000,2000)
# ax2.set_xlim(0,3000)
# ax.set_xlim(150,200)
plt.savefig(os.path.join(folder,'overlap_wm_avgcts_plot.png'))

# Analyse measure power tracking

In [ ]:
folder= tb.latest_data('power')
d = np.loadtxt(tb.get_msmt_fp(folder, ext='dat'), skiprows=12)
time, power = d[:,0],d[:,1]

In [ ]:
fig,ax = plt.subplots()
ax.plot((time-time[0])/60,power*1.e6)
ax.set_xlabel('time (minutes)',fontsize=14)
ax.set_ylabel('power (uW)',fontsize=14)
ax.set_title(folder)
plt.savefig(os.path.join(folder,'plot.png'))


# analyse data tracked by PID cavity length

In [ ]:
folder= tb.latest_data('pid')#r'D:\measuring\data\20171207\135353_pidcavl'
#r'D:\measuring\data\20171207\133039_pidcavl'#r'D:\measuring\data\20171207\130217_pidcavl'#r'D:\measuring\data\20171206\171035_pidcavl'
d = np.loadtxt(tb.get_msmt_fp(folder, ext='dat'), skiprows=23)


modulation_freq = 200 #Hz
modulation_bins = 200

time, raw_error_signal, avg_error_signal, setpoint, control_param, avg_cts, wavemeter_freq = d[:,0],d[:,1],d[:,2],d[:,3],d[:,4],d[:,5],d[:,6]


# time = time + 17*3600+51*60+8

In [ ]:
nr_wm_bins = 3
timeCut = time[len(time)%nr_wm_bins:]
wavemeter_freqCut = wavemeter_freq[len(time)%nr_wm_bins:]
timeBinned = np.mean(timeCut.reshape(-1, nr_wm_bins), axis=1)
wavemeter_freqBinned =np.mean(wavemeter_freqCut.reshape(-1, nr_wm_bins), axis=1)

avg_ctsCut = avg_cts[len(time)%nr_wm_bins:]
avg_ctsBinned =np.mean(avg_ctsCut.reshape(-1, nr_wm_bins), axis=1)


In [ ]:
N=50
wavemeter_freqBinned2 = np.convolve(wavemeter_freqBinned, np.ones((2*N,))/(2*N), mode='valid')


In [ ]:
fig,ax=plt.subplots(1, figsize=(16,5))
ax.set_title(folder)
ax.plot(timeBinned/60,avg_ctsBinned*modulation_freq,'-',color='red')
ax.set_xlabel("time (minutes)", fontsize = 14)
ax.set_ylabel("Average countrate (Hz)", fontsize = 14, color ='r')
ax2 =ax.twinx()
ax.set_ylim(0,300000)
# ax.set_xlim(6,11)
# ax2.set_xlim(6,11)
ax2.plot(time/60,control_param)
ax2.set_ylabel('newfocus freqmod (V)', color = 'b',fontsize=14)
# ax.set_xlim(150,200)
plt.savefig(os.path.join(folder,'overlap_wm_avgcts_plot.png'))

In [ ]:
fig,ax=plt.subplots(1, figsize=(16,5))
ax.set_title(folder)
# ax.plot(timeBinned/60,(wavemeter_freqBinned), '-b')
ax.plot(time/60, wavemeter_freq,'-m')
ax.set_ylabel('wavemeter readout (GHz)',fontsize=14)
ax.set_xlabel('time(minutes)',fontsize=14)
plt.savefig(os.path.join(folder,'plot_wavemeter_ro.png'))
# ax.set_xlim(150,200)
plt.show()


In [ ]:
fig,ax=plt.subplots(1, figsize=(16,5))
ax.set_title(folder)
ax.plot(time/60,avg_error_signal,'-',color='red')
ax.set_xlabel("time (minutes)", fontsize = 14)
ax.set_ylabel("Average error signal", fontsize = 14, color ='r')
ax2 =ax.twinx()
ax.set_ylim(-3000,3000)
# ax.set_xlim(6,11)
# ax2.set_xlim(6,11)
ax2.plot(time/60,control_param)
ax2.set_ylabel('jpe voltage (V)', color = 'b',fontsize=14)
# ax.set_xlim(150,200)
plt.savefig(os.path.join(folder,'overlap_wm_avgcts_plot.png'))

In [ ]:
int_times=np.linspace(5,5,181)
equals = np.argwhere(np.diff(control_param)==0)

plt.plot(equals)
plt.plot(np.diff(equals,axis=0))
# plt.xlim(0,25)
# plt.plot(np.argwhere(np.diff(equals,axis=0)>50))
steps = np.argwhere(np.diff(equals[:,0],axis=0)>1)
steps2=np.append(steps[np.argwhere(np.diff(steps[:,0],axis=0)>1)][:,0],steps[-1])

for i in steps2:
    plt.plot([i,i],[0,9000])
plt.show()

cp_steps = control_param[steps2]
fig,ax = plt.subplots()
ax.plot(int_times,cp_steps,'o')# print cp_steps
plt.show()

fig,ax = plt.subplots()
ax.plot(int_times[1:],np.diff(cp_steps),'o')# print c\p_steps
plt.show()



In [ ]:
g_a0 = 0
g_a1 = 2
g_a2 = 1
fixed=[]

p0, fitfunc, fitfunc_str = common.fit_poly(g_a0,g_a1,g_a2)
fit_res = fit.fit1d(time[steps2]/60,(cp_steps-cp_steps[0])*1000, fitmethod=None, p0=p0, fitfunc=fitfunc, fitfunc_str=fitfunc_str,fixed=fixed)
ax=plot.plot_fit1d(fit_res,ret='ax')
ax.set_ylabel('absolute JPE voltage change (mV)')
ax.set_xlabel('wait time after stabilisation (minutes)')

print fit_res['params_dict']['a2']



In [ ]:
L=13*2.41+3.6 #in um
nu = 470.4e3#+wavemeter_freq[0]
conversion = 65e-6 #um/mV
dL = wavemeter_freq/nu*L
dnu_steps = (cp_steps-cp_steps[0])*1000*conversion/L*nu #in GHz

g_a0 = 0
g_a1 = 2
fixed=[]

print len(dnu_steps)
times = time[equals[steps2[:]][:,0]]


p0, fitfunc, fitfunc_str = common.fit_poly(g_a0,g_a1)
fit_res = fit.fit1d(times/60,(dnu_steps), fitmethod=None, p0=p0, fitfunc=fitfunc, fitfunc_str=fitfunc_str,fixed=fixed)
ax=plot.plot_fit1d(fit_res,ret='ax')
ax.set_ylabel('effective frequency change (GHz)')
ax.set_xlabel('time after stabilisation (minutes)')
ax.set_title(folder)
plt.savefig(os.path.join(folder,'freq_drift_during_free_evolution.png'))

print 'total drift:',round(fit_res['params_dict']['a1'],2)*(times[-1]-times[0])/60,'GHz'
print 'total drift per second:',round(fit_res['params_dict']['a1']*1000,2),'MHz per second'


In [ ]:
L=13*2.41+3.6 #in um
nu = 470.4e3-273#+wavemeter_freq[0]
conversion = 65e-6 #um/mV
dL = wavemeter_freq/nu*L
dnu_steps = (cp_steps-cp_steps[0])*1000*conversion/L*nu #in GHz


g_a0 = 0
g_a1 = 2
fixed=[]
total_time = np.zeros(len(int_times))

for i,t in enumerate(int_times):
    if i==0:
        total_time[i] = t
    else:
        total_time[i] = t+total_time[i-1]

p0, fitfunc, fitfunc_str = common.fit_poly(g_a0,g_a1)
fit_res = fit.fit1d(total_time[1:-2],(dnu_steps)[1:-2], fitmethod=None, p0=p0, fitfunc=fitfunc, fitfunc_str=fitfunc_str,fixed=fixed)
ax=plot.plot_fit1d(fit_res,ret='ax')
ax.set_ylabel('effective frequency change (GHz)')
ax.set_xlabel('wait time after stabilisation (s)')
ax.set_title(folder)
plt.savefig(os.path.join(folder,'freq_drift_during_free_evolution.png'))

print 'drift per second during free evolution:',round(fit_res['params_dict']['a1']*1000,2),'MHz per second'



In [ ]:

g_a0 = 0
g_a1=-0.06
fixed=[]

p0, fitfunc, fitfunc_str = common.fit_poly(g_a0,g_a1)
fit_res = fit.fit1d(int_times[3:],np.diff(cp_steps[2:])*1000, fitmethod=None, p0=p0, fitfunc=fitfunc, fitfunc_str=fitfunc_str,fixed=fixed)
ax=plot.plot_fit1d(fit_res,ret='ax')
ax.set_ylabel('relative JPE voltage change (mV)')
ax.set_xlabel('wait time after stabilisation (s)')

print 'a0',fit_res['params_dict']['a0']
mV_drift_per_sec = fit_res['params_dict']['a1']
MHz_drift_per_sec = mV_drift_per_sec*conversion/L*nu*1000
print 'drift per second (microV): ',round(mV_drift_per_sec*1000,1)

print 'drift per second (MHz)' , round(MHz_drift_per_sec,2) #in GHz



In [ ]:
fig,ax=plt.subplots(1, figsize=(16,5))
ax.set_title(folder)
ax.plot(timeBinned/60,(wavemeter_freqBinned+285), '-b')
ax.plot(timeBinned[N-1:-N]/60,(wavemeter_freqBinned2+285), '-g')
ax.set_ylabel('wavemeter readout + 285 GHz',fontsize=14)
ax.set_xlabel('time(minutes)',fontsize=14)
plt.savefig(os.path.join(folder,'plot_wavemeter_ro.png'))
plt.show()


In [ ]:

fig,ax=plt.subplots(1, figsize=(16,5))
ax.set_title(folder)
ax2 = ax.twinx()
ax.plot(timeBinned[N-1:-N]/60,(wavemeter_freqBinned[N-1:-N]-wavemeter_freqBinned2), '-b') #use smooth wavemeter data for plotting derivative.
ax2.plot(timeBinned/60.,avg_ctsBinned*modulation_freq,'-',color='red')
ax.set_ylabel("trend-corrected wavemeter readout", fontsize = 14,color='b'), 
ax2.set_xlabel("time (miutes)", fontsize = 14)
ax.set_ylim(-0.02,0.05)
# ax2.set_ylim(min(Sample_TempBinned_Diff),max(Sample_TempBinned_Diff))
# ax3 = ax.twinx()
# ax3.plot(TimeBinned[1:]/60,abs(np.diff(Sample_TempBinned_Diff)), '-b') #use smooth wavemeter data for plotting derivative.
# ax3.spines['right'].set_position(('axes', 1.+0.08))
# ax3.set_ylabel("derivative wavemeter readout ", fontsize = 14,color='b'), 
# # ax2.set_ylim(min(Sample_TempBinned_Diff),max(Sample_TempBinned_Diff))

ax2.set_ylabel("Average countrate (Hz)", fontsize = 14, color ='r')
ax.set_xlabel('time (minutes)', fontsize = 14)
ax.set_xlim(min(time/60),max(time/60))
plt.savefig(os.path.join(folder,'overlap_wm_avgcts_plot.png'))

In [ ]:
fig,ax=plt.subplots(1, figsize=(16,5))
plt.plot(time/60.,raw_error_signal,'-',color='red')
ax.set_title(folder)
ax.set_xlabel("time (minutes)", fontsize = 14)
ax.set_ylabel("error signal", fontsize = 14)
ax.set_xlim(min(time/60),max(time/60))
plt.savefig(os.path.join(folder,'error_signal_plot.png'))

In [ ]:
fig,ax=plt.subplots(1, figsize=(16,5))
plt.plot(time/60.,wavemeter_freq,'-',color='red')
ax.set_title(folder)
ax.set_xlabel("time (minutes)", fontsize = 14)
ax.set_ylabel("wavemeter readout (GHz)", fontsize = 14)
ax.set_xlim(min(time/60),max(time/60))
plt.savefig(os.path.join(folder,'wm_freq_plot.png'))

## correlate with temperature data


In [ ]:
temp_d0 = np.loadtxt(folder+'\MI_TempDataLog 12_07_2017.csv', skiprows=1, 
                    delimiter = ',',usecols = (1,2,4), 
                    converters = {1: lambda r: int(r[1:3])*3600+int(r[4:6])*60+int(r[7:9])+int(r[10:13])/1000., 
                                  2: lambda s: int(s[1:3])*60+int(s[4:6])+int(s[7:9])/60.} ) ##,
temp_d1 = np.loadtxt(folder+'\MI_TempDataLog 12_08_2017.csv', skiprows=1, 
                    delimiter = ',',usecols = (1,2,4), 
                    converters = {1: lambda r: int(r[1])*3600+int(r[3:5])*60+int(r[6:8])+int(r[9:12])/1000., 
                                  2: lambda s: int(s[1:3])*60+int(s[4:6])+int(s[7:9])/60.} ) ##,

Time0, Relative_Time0,Sample_Temp0 = temp_d0[:,0], temp_d0[:,1], temp_d0[:,2]
Time1, Relative_Time1,Sample_Temp1 = temp_d1[:,0]+3600*24, temp_d1[:,1], temp_d1[:,2]

Time = np.append(Time0,Time1)-18*60 #cryo is 18 minutes ahead in time
Relative_Time = np.append(Relative_Time0,Relative_Time1)
Sample_Temp = np.append(Sample_Temp0, Sample_Temp1)

In [ ]:
nr_bins = 30 #temperature is recorded every 2 seconds -> bin to give the average per minute
TimeCut = Time[len(Time)%nr_bins:]
Sample_TempCut = Sample_Temp[len(Time)%nr_bins:]
TimeBinned = np.mean(TimeCut.reshape(-1, nr_bins), axis=1)
Sample_TempBinned =np.mean(Sample_TempCut.reshape(-1, nr_bins), axis=1)



plt.figure()
plt.plot(Relative_Time0, Sample_Temp0)
plt.plot(Relative_Time1, Sample_Temp1, 'c')
plt.show()
plt.figure()
plt.plot(Time0/60, Sample_Temp0)
plt.plot(Time1/60, Sample_Temp1, 'm')
plt.show()
plt.figure()
plt.plot(TimeBinned, Sample_TempBinned )
plt.show()

plt.figure()
# plt.plot(time, wavemeter_freq )
plt.plot(timeBinned, wavemeter_freqBinned )
plt.show()

# plt.plot(Relative_Time, Sample_Temp)

In [ ]:

init_sample_temp =Sample_TempBinned[np.argmin(abs(TimeBinned-time[0]))]
Sample_TempBinned_Diff=(Sample_TempBinned-init_sample_temp)*1000 # in mK
init_jpe_V = control_param[0]
control_param_diff = (control_param - init_jpe_V)*1000 #in mV
init_wavemeter = wavemeter_freq[0]
wavemeter_freq_diff = (wavemeter_freq - init_wavemeter)

L=13*2.41+3.6 #in um
nu = 470.4e3+wavemeter_freq[0]
conversion = 65e-6 #um/mV
dL = wavemeter_freq/nu*L
dnu = control_param_diff*conversion/L*nu #in GHz


fig,ax=plt.subplots(1, figsize=(16,5))


ax.plot(TimeBinned/60,Sample_TempBinned_Diff, '-c')
ax.set_ylabel("sample temperature - %.3f (mK)"%(init_sample_temp), fontsize = 14,color='c')
ax.set_ylim(min(Sample_TempBinned_Diff),max(Sample_TempBinned_Diff))


# ax3 = ax.twinx()
# ax3.plot(time/60,wavemeter_freq_diff, 'm')
# ax3.spines['right'].set_position(('axes', 1.+0.08))
# ax3.set_ylabel('wavemeter readout - %.0f GHz (GHz)'%(init_wavemeter), fontsize=14,color='m')
# ax3.set_ylim(min(wavemeter_freq_diff),max(wavemeter_freq_diff))

ax4 = ax.twinx()
ax4.plot(time/60,dnu-wavemeter_freq_diff, 'r')
ax4.set_ylabel('laser freq-corrected resonance drift (GHz)', fontsize=14,color='r')
ax4.set_ylim(min(dnu),max(dnu))

# ax2 = ax.twinx()
# ax2.spines['right'].set_position(('axes', 1.+0.08))
# ax2.plot(time/60.,control_param_diff,'-',color='pink')
# ax2.set_ylabel("jpe piezo voltage - %.3f V (mV)"%(init_jpe_V), fontsize = 14, color = 'pink')
# ax2.set_ylim(min(control_param_diff),max(control_param_diff))


ax.set_title(folder)
ax.set_xlabel("time (minutes)", fontsize = 14)
ax.set_xlim(min(time/60),max(time/60))
plt.savefig(os.path.join(folder,'plot_piezo_temp_vs_time.png'))
# plt.savefig(os.path.join(folder,'plot_all_vs_time.png'))


In [ ]:
fig,ax=plt.subplots(1, figsize=(16,5))
plt.plot(time/60.,avg_cts*modulation_freq,'-',color='red')
ax.set_title(folder)
ax2 = ax.twinx()
ax2.plot(TimeBinned[1:]/60,np.diff(Sample_TempBinned_Diff), '-c')
ax2.set_ylabel("sample temperature - %.3f (mK)"%(init_sample_temp), fontsize = 14,color='c')
# ax2.set_ylim(min(Sample_TempBinned_Diff),max(Sample_TempBinned_Diff))

ax.set_xlabel("time (miutes)", fontsize = 14)
ax.set_ylabel("Countrate (Hz)", fontsize = 14)
ax.set_xlim(min(time/60),max(time/60))
plt.savefig(os.path.join(folder,'plot.png'))